In [58]:
# Import dependencies
import pandas as pd
import os
from dotenv import load_dotenv    # from Karen's code
from sqlalchemy import create_engine
from sqlalchemy import text
from sqlalchemy import select      # Not used
import psycopg2
import csv   # Not used
import numpy as np   # Not used
from pprint import pprint
import json


# import gzip    # Not used

In [59]:

url_metar_gz="https://aviationweather.gov/data/cache/metars.cache.csv.gz"
url_TAF_gz="https://aviationweather.gov/data/cache/tafs.cache.csv.gz"
url_airsigmets_gz="https://aviationweather.gov/data/cache/airsigmets.cache.csv.gz"

__To retrieve METAR data__

In [60]:
# Fetch, load, and decompress the data relative to metar
metar_data_df = pd.read_csv(url_metar_gz, header=5, compression='gzip')
metar_data_df.head()

,raw_text,station_id,observation_time,latitude,longitude,temp_c,dewpoint_c,wind_dir_degrees,wind_speed_kt,wind_gust_kt,...,maxT24hr_c,minT24hr_c,precip_in,pcp3hr_in,pcp6hr_in,pcp24hr_in,snow_in,vert_vis_ft,metar_type,elevation_m
0,PFEL 280817Z AUTO 19008KT 10SM SCT011 OVC018 M...,PFEL,2023-12-28T08:17:00Z,64.6160,-162.2720,-13.0,-15.0,190,8.0,NaN,...,NaN,NaN,0.005,NaN,NaN,NaN,NaN,NaN,SPECI,50.0
1,KSGH 280817Z AUTO 26003KT 4SM BR SCT006 OVC028...,KSGH,2023-12-28T08:17:00Z,39.8358,-83.8443,4.0,4.0,260,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SPECI,318.0
2,KPVG 280817Z VRB03KT 2 1/2SM OVC003 14/14 A296...,KPVG,2023-12-28T08:17:00Z,36.7824,-76.4520,14.0,14.0,VRB,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SPECI,7.0
3,KNYG 280817Z AUTO 32004KT 3SM RA BR BKN005 OVC...,KNYG,2023-12-28T08:17:00Z,38.5030,-77.3010,11.7,10.6,320,4.0,NaN,...,NaN,NaN,0.010,NaN,NaN,NaN,NaN,NaN,SPECI,2.0
4,KBTP 280817Z AUTO VRB03KT 10SM SCT005 OVC035 0...,KBTP,2023-12-28T08:17:00Z,40.7765,-79.9568,8.0,8.0,VRB,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SPECI,375.0


In [61]:
# Convert the dataframe to a JSON format
# metar_json = json.loads(json.dumps(list(metar_data_df.T.to_dict().values())))   # https://stackoverflow.com/questions/39257147/convert-pandas-dataframe-to-json-format answer #10 Amir.S
# pprint(metar_json, sort_dicts=False)  # the pprint process is slow (10.5sec)

In [62]:
# pprint(metar_json, sort_dicts=False)

In [63]:
# Convert some columns to INT
# cols=['wind_dir_degrees','wind_speed_kt','wind_gust_kt','cloud_base_ft_agl','cloud_base_ft_agl.2',\
#     'cloud_base_ft_agl.3','vert_vis_ft','elevation_m']
# metar_data_df[cols]=metar_data_df[cols].apply(pd.to_numeric, errors='coerce', downcast='integer', axis=1) # Does not appear to convert to INT, but to FLOAT64
# metar_data_df[cols]=metar_data_df[cols].astype(int)   # Cannot convert NaN

In [64]:
# metar_data_df.columns

In [65]:
# metar_data_df.info()

In [66]:
# convert the dataframe to a dictionary then to a JSON string
metar_string=json.dumps(list(metar_data_df.T.to_dict().values()))

# open the file in write mode
# output_path = os.path.join("Resources", "metar_data.json")  # To be removed if logic.js cannot read from Resources
output_path = os.path.join("", "metar_data.json")
with open(output_path, "w") as file:
    # write the JSON string to the file
    file.write(metar_string.replace("NaN","null"))

# file is automatically closed after the with block

In [67]:
# To save the dataframe as a csv file for future import to database (not needed anymore)
# output_path2 = os.path.join("", "metar_data.csv")
# metar_data_df.to_csv(output_path2, index=False)

# file is automatically closed after the with block

__To refresh the metar table in the Render database__

In [68]:
# Test to refresh the metar table

load_dotenv()
db_url = os.environ.get("link_render")
connection = psycopg2.connect(db_url)
cursor = connection.cursor()
cursor.execute("DELETE FROM metar;")    # Deletes all the rows but keep the table
connection.commit()
cursor.close()
connection.close()

In [69]:
# Test to populate the empty table from a json file
load_dotenv()
db_url = os.environ.get("link_render")
connection = psycopg2.connect(db_url)
cursor = connection.cursor()

cursor.execute("set search_path to public") # https://dba.stackexchange.com/questions/268365/using-python-to-insert-json-into-postgresql

with open('metar_data.json') as file:
    data = file.read()

query_sql = """
INSERT INTO metar SELECT * FROM
json_populate_recordset(NULL::metar, %s);
"""

cursor.execute(query_sql, (data,))
connection.commit()

__To download a new csv file made of the joining of airport data and weather data__

In [70]:
# download the output of a query across multiple tables with all the all the active public airport with or without weather information.
# Will be used to populate the makers that do not have METAR information.
# There is only one row per airport so only one runway is listed even if the airport as more.
# More parameters can be added to complete the info on the popup window.

load_dotenv()
db_url = os.environ.get("link_render")

# query="""
#     SELECT DISTINCT ON (arpt_id)
# 	arpt_id, icao_id, metar.station_id, arpt_name, apt_rwy.rwy_id, lat_decimal, long_decimal, metar.observation_time, metar.wind_speed_kt, metar.flight_category, metar.raw_text
#     FROM apt_rwy
#     JOIN apt_base ON apt_base.site_no = apt_rwy.site_no
#     FULL JOIN metar ON metar.station_id = apt_base.icao_id
#     WHERE facility_use_code='PU' AND site_type_code='A' AND arpt_status='O';
#     """

query="""
    SELECT DISTINCT ON (arpt_id)
	arpt_id, icao_id, metar.station_id, arpt_name, apt_rwy.rwy_id, lat_decimal, long_decimal, metar.observation_time, metar.wind_speed_kt, metar.flight_category, metar.raw_text
    FROM apt_rwy
    JOIN apt_base ON apt_base.site_no = apt_rwy.site_no
    FULL JOIN metar ON (RIGHT(metar.station_id, LENGTH(metar.station_id) - 1)) = apt_base.arpt_id
    WHERE facility_use_code='PU' AND site_type_code='A' AND arpt_status='O'
    """

engine=create_engine(db_url)
with engine.begin() as conn:
    results=conn.execute(
        text(query)
    )

arpt_weather_query_df = pd.DataFrame(results)

# Save the df as a CSV file. Might not be needed if we only use JSON    TO BE REMOVED?
# arpt_weather_query_df.to_csv (r'airport_weather_data.csv', index = False) # place 'r' before the path name


# convert the dataframe to a dictionary then to a JSON string
arpt_weather_string=json.dumps(list(arpt_weather_query_df.T.to_dict().values()))

# open the file in write mode
# output_path = os.path.join("Resources", "metar_data.json")  # To be removed if logic.js cannot read from Resources
output_path = os.path.join("", "airport_weather_data.json")
with open(output_path, "w") as file:
    # write the JSON string to the file
    file.write(arpt_weather_string.replace("NaN","null"))

# file is automatically closed after the with block



In [71]:
# arpt_weather_query_df.head()

In [72]:
# pprint(arpt_weather_string)

__To retrieve TAF data__

In [73]:
# Fetch, load, and decompress the data relative to TAF  (working but not used for now)
# taf_data_df = pd.read_csv(url_TAF_gz, header=5,index_col=False, compression='gzip',dtype='str',low_memory=False)
# taf_data_df.head()

In [74]:
# Convert the dataframe to a JSON format  (working but not used for now)
# taf_json = json.loads(json.dumps(list(taf_data_df.T.to_dict().values())))   # https://stackoverflow.com/questions/39257147/convert-pandas-dataframe-to-json-format answer #10 Amir.S
# pprint(taf_json, sort_dicts=False)  # the pprint process is slow (10.5sec)

In [75]:
# # convert the dataframe to a dictionary then to a JSON string  (working but not used for now)
# taf_string=json.dumps(list(taf_data_df.T.to_dict().values()))

# # open the file in write mode
# output_path = os.path.join("Resources", "taf_data.json")
# with open(output_path, "w") as file:
#     # write the JSON string to the file
#     file.write(taf_string.replace("NaN","null"))

# # file is automatically closed after the with block

__To retrieve AIRMET and SIGMET polygons data__

In [76]:
# Fetch, load, and decompress the data relative to Airmet and Sigmet
airsigmet_data_df = pd.read_csv(url_airsigmets_gz, header=5, compression='gzip', encoding='utf-8')
airsigmet_data_df.head()

,raw_text,valid_time_from,valid_time_to,lon:lat points,min_ft_msl,max_ft_msl,movement_dir_degrees,movement_speed_kt,hazard,severity,airsigmet_type
0,WSUS03 KKCI 280640 CHIV WS 280640 SIGMET VIC...,2023-12-28T06:40:00Z,2023-12-28T10:40:00Z,-80.3491:39.3845;-74.1995:39.291;-75.7532:37.2...,27000.0,40000.0,NaN,NaN,TURB,MOD-SEV,SIGMET
1,WSUS01 KKCI 280648 BOSR WS 280648 SIGMET ROM...,2023-12-28T06:48:00Z,2023-12-28T10:48:00Z,-66.9353:41.2385;-67.0108:38.8895;-72.4317:37....,26000.0,37000.0,NaN,NaN,TURB,MOD-SEV,SIGMET
2,WSUS31 KKCI 280755 SIGE CONVECTIVE SIGMET 1...,2023-12-28T07:55:00Z,2023-12-28T09:55:00Z,-73.1709:42.2947;-72.9857:40.9181;-74.54:40.89...,30000.0,NaN,NaN,CONVECTIVE,LT-MOD,SIGMET,NaN
3,WSUS31 KKCI 280755 SIGE CONVECTIVE SIGMET 1...,2023-12-28T07:55:00Z,2023-12-28T09:55:00Z,-82.8062:26.1447;-82.4528:25.1793;-82.4466:23....,27000.0,NaN,NaN,CONVECTIVE,LT-MOD,SIGMET,NaN
4,WSUS32 KKCI 280755 SIGC CONVECTIVE SIGMET.....,2023-12-28T07:55:00Z,2023-12-28T09:55:00Z,-107:49;-107:31.79;-106.52:31.78;-106.48:31.75...,NaN,NaN,NaN,NaN,CONVECTIVE,1,SIGMET


In [77]:
# replace with <br> in raw_text
# airsigmet_data_df['raw_text'].str.replace(r' \x07','<br>')
# airsigmet_data_df.replace(r' x07','<br>', regex=True)

# To convert the points delimiting the areas into something Leaflet-friendly
for j in range(len(airsigmet_data_df)):
    test=airsigmet_data_df.iloc[j]['lon:lat points']
    if  pd.isna(test)!=True:    # Test if the cell is not NaN
        test1=test.split(';')
        for i in range(len(test1)):
            test1[i]=test1[i].split(':')    # Creates list of coordinates
            test1[i][0],test1[i][1]=float(test1[i][1]),float(test1[i][0])   # Swap lon:lat to lat:lon
        airsigmet_data_df.at[j,'lon:lat points']=test1

    # shift cells for missing column
    test2=airsigmet_data_df.iloc[j]['airsigmet_type']
    if  pd.isna(test2)==True:    # Test if the cell is NaN
        airsigmet_data_df.at[j,'airsigmet_type']=airsigmet_data_df.at[j,'severity']
        airsigmet_data_df.at[j,'severity']=airsigmet_data_df.at[j,'hazard']
        airsigmet_data_df.at[j,'hazard']=airsigmet_data_df.at[j,'movement_speed_kt']
        airsigmet_data_df.at[j,'movement_speed_kt']=airsigmet_data_df.at[j,'movement_dir_degrees']
        airsigmet_data_df.at[j,'movement_dir_degrees']=airsigmet_data_df.at[j,'max_ft_msl']
        airsigmet_data_df.at[j,'max_ft_msl']=airsigmet_data_df.at[j,'min_ft_msl']
        airsigmet_data_df.at[j,'min_ft_msl']="NaN"

  


airsigmet_data_df.rename(columns={"lon:lat points":"lat_lon_points"}, inplace=True) # Update the column name
airsigmet_data_df.head()

,raw_text,valid_time_from,valid_time_to,lat_lon_points,min_ft_msl,max_ft_msl,movement_dir_degrees,movement_speed_kt,hazard,severity,airsigmet_type
0,WSUS03 KKCI 280640 CHIV WS 280640 SIGMET VIC...,2023-12-28T06:40:00Z,2023-12-28T10:40:00Z,"[[39.3845, -80.3491], [39.291, -74.1995], [37....",27000.0,40000.0,NaN,NaN,TURB,MOD-SEV,SIGMET
1,WSUS01 KKCI 280648 BOSR WS 280648 SIGMET ROM...,2023-12-28T06:48:00Z,2023-12-28T10:48:00Z,"[[41.2385, -66.9353], [38.8895, -67.0108], [37...",26000.0,37000.0,NaN,NaN,TURB,MOD-SEV,SIGMET
2,WSUS31 KKCI 280755 SIGE CONVECTIVE SIGMET 1...,2023-12-28T07:55:00Z,2023-12-28T09:55:00Z,"[[42.2947, -73.1709], [40.9181, -72.9857], [40...",NaN,30000.0,NaN,NaN,CONVECTIVE,LT-MOD,SIGMET
3,WSUS31 KKCI 280755 SIGE CONVECTIVE SIGMET 1...,2023-12-28T07:55:00Z,2023-12-28T09:55:00Z,"[[26.1447, -82.8062], [25.1793, -82.4528], [23...",NaN,27000.0,NaN,NaN,CONVECTIVE,LT-MOD,SIGMET
4,WSUS32 KKCI 280755 SIGC CONVECTIVE SIGMET.....,2023-12-28T07:55:00Z,2023-12-28T09:55:00Z,"[[49.0, -107.0], [31.79, -107.0], [31.78, -106...",NaN,NaN,NaN,NaN,CONVECTIVE,1,SIGMET


In [78]:
# Convert the dataframe to a JSON format
# airsigmet_json = json.loads(json.dumps(list(airsigmet_data_df.T.to_dict().values())))   # https://stackoverflow.com/questions/39257147/convert-pandas-dataframe-to-json-format answer #10 Amir.S
# pprint(airsigmet_json, sort_dicts=False)

In [79]:
# convert the dataframe to a dictionary then to a JSON string
airsigmet_string=json.dumps(list(airsigmet_data_df.T.to_dict().values()))

# open the file in write mode
# output_path = os.path.join("Resources", "airsigmet_data.json")
output_path = os.path.join("", "airsigmet_data.json")
with open(output_path, "w") as file:
    # write the JSON string to the file
    file.write(airsigmet_string.replace("NaN","null"))

# file is automatically closed after the with block